- BERT 계열과 GPT 계열의 공통 motivation, 차이점
- LLM 의 대표적인 2가지: BERT, GPT
- In-context inference

1. GPT 모델 전이학습 (추가학습)

비지도 학습과 생성적 사전학습(generative pre-training)기법, 변환기(transformer)를 적용

#pip install tensorflow
#git clone https://github.com/openai/gpt-2.git
#cd gpt-2
python download_model.py


In [ ]:
import tensorflow as tf
import os
import sys
import numpy as np

sys.path.append('gpt-2/src')

import model, sample, encoder

model_name = '117M'  # 모델 크기 선택 (117M 또는 345M)
seed = None
length = 100  # 생성할 텍스트 길이

models_dir = 'gpt-2/models'
model_dir = os.path.join(models_dir, model_name)

if seed is None:
    seed = np.random.randint(0, 1000000)

enc = encoder.get_encoder(model_name, models_dir)
hparams = model.default_hparams()
with open(os.path.join(model_dir, 'hparams.json')) as f:
    hparams.override_from_dict(json.load(f))

with tf.Session(graph=tf.Graph()) as sess:
    context = tf.placeholder(tf.int32, [1, None])
    np.random.seed(seed)
    tf.set_random_seed(seed)
    output = sample.sample_sequence(
        hparams=hparams, length=length,
        context=context,
        batch_size=1,
        temperature=1.0
    )

    saver = tf.train.Saver()
    ckpt = tf.train.latest_checkpoint(os.path.join(model_dir, 'checkpoint'))
    saver.restore(sess, ckpt)

    raw_text = "Your prompt goes here."  # 여기에 사용하고자 하는 프롬프트 입력
    context_tokens = enc.encode(raw_text)
    out = sess.run(output, feed_dict={context: [context_tokens]})

    text = enc.decode(out[0])
    print(text)

In [ ]:
#gpt

2. NLP 딥러닝 모델 훈련